## exceptions

All programming languages ​​provide some way to deal with unexpected errors. Whether it is a file that does not exist, a web page that cannot be reached, a division by zero, these are all errors that _a priori_ could happen and that the developer should consider when designing the code. There is no more infuriating user experience than a program closing itself without any explanation or possibility of user intervention.

While we'll cover some more appropriate ways to handle error situations later, the traditional way is through *exception* handling. The exception is the signal of an abnormal situation that interrupts the normal flow of the program. In this case, an attempt is made to catch the exception and proceed accordingly.

In [1]:
let getHead (l: int list) = 
    l.Head 

getHead []

Error: System.InvalidOperationException: The input list was empty.
   at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4072
   at <StartupCode$FSI_0005>.$FSI_0005.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In this case the system throws the exception `InvalidOperationException`, and tells us the cause `The input list was empty`.

How do we catch the exception? With a `try...with` block:

In [2]:
let getHeadExn (l: int list) = 
    try 
        l.Head 
    with
    | :? System.InvalidOperationException  -> 
        printfn "Exception!" 
        -1 

getHeadExn []

Exception!


-1

Again F# uses _pattern matching_ to check for the _type_ of exception, via the `:?` symbol. The `try...with` block is still an expression, so both branches (`try` and `with`) have to return the same data type. Hence the value `-1` in the `with` branch, to indicate that it is not possible to get the `Head` value from the list.

It is possible to obtain more data from the exception, as follows:


In [14]:
let getHeadExnCustomMsg (l: int list) = 
    try 
        l.Head 
    with
    | :? System.InvalidOperationException as ex  -> 
        printfn "Exception!: %A" (ex.Message)
        -1 

getHeadExnCustomMsg []

Exception!: "The input list was empty."


-1

Exceptions can be created instead of using system ones:

In [15]:
exception ExceptionForList of string 

let getHeadExn2 (l: int list) = 
    try 
        match l with 
        | [] -> raise (ExceptionForList "List is empty")
        | _ -> l.Head 
    with
    | ExceptionForList s  -> 
        printfn "%s" s 
        -1 
        

getHeadExn2 []

List is empty


-1

In this case it is the programmer who decides to throw a custom exception, through the `raise` function.

In [3]:
exception DivideByOne of string 

let divide x y =
    match y with 
    | 0 -> invalidOp "Trying to divide by zero!!" 
    | 1 -> raise (DivideByOne "Can't believe you are trying to divide by one")
    | _ -> x/y 

printfn "%A" (divide 4 2)

2


In [4]:
printfn "%A" (divide 4 1)


Error: FSI_0016+DivideByOne: DivideByOne "Can't believe you are trying to divide by one"
   at FSI_0016.divide(Int32 x, Int32 y)
   at <StartupCode$FSI_0017>.$FSI_0017.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [5]:
printfn "%A" (divide 4 0)

Error: System.InvalidOperationException: Trying to divide by zero!!
   at FSI_0016.divide(Int32 x, Int32 y)
   at <StartupCode$FSI_0018>.$FSI_0018.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [7]:
let annoyedDivide x y =
    try 
        divide x y 
    with 
    | DivideByOne s -> 
        printfn "%A" (s)
        -1
    | :? System.InvalidOperationException as ex -> 
        printfn "Message: %A" (ex.Message)
        -1          

        

In [8]:
annoyedDivide 4 2 


2

In [9]:
annoyedDivide 4 1 

"Can't believe you are trying to divide by one"


-1

In [10]:
annoyedDivide 4 0

Message: "Trying to divide by zero!!"


-1

The `annoyedDivide` function catches exceptions, and in those cases returns `-1`.

> Note that it is at least moot that a function that divides would return -1 as a value on error, since -1 is effectively a valid value when doing a division. Although the program will continue to run since the exception was caught, the problem is now with the function that calls `annoyedDivide`, since it has to distinguish between the case of -1 as a valid value or -1 as an error sign. We will see later a robust way to proceed in this case.
